In [ ]:
!pip3 install -U selenium
!pip3 install webdriver-manager

In [ ]:
# Chrome -> chromedriver
# firefox -> geckodriver

In [ ]:
# passo 1: pegar a cotacao dolar
# passo 2: pegar a cotacao euro
# passo 3: pegar a cotacao ouro
# passo 4: atualizar a base de dados
# passo 5: recalcular os precos
# passo 6: exportar a base de dados

In [58]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# passo 1: pegar a cotacao dolar
# abrir o navegador
driver = webdriver.Chrome()

# abrir o google
driver.get('https://www.google.com.br/')

# pesquisar a cotacao do dolar
driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

dolar_value = driver.find_element('xpath',
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(dolar_value)


5.160299999999999


In [59]:
# abrir o google
driver.get('https://www.google.com.br/')

# passo 2: pegar a cotacao euro
driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

euro_value = driver.find_element('xpath',
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


print(euro_value)

5.257133029


In [60]:
# abrir o site com cotação do ouro
driver.get('https://www.melhorcambio.com/ouro-hoje')

# passo 3: pegar a cotacao ouro
gold_value = driver.find_element('xpath', 
                    '//*[@id="comercial"]').get_attribute('value')

print(gold_value)

290,90


In [61]:
# transformar para o formato de concatenação do python
gold_value = gold_value.replace(',','.')
print(gold_value)

# Fechar o navegador 
driver.quit()

290.90


In [62]:
import pandas as pd

# passo 4: atualizar a base de dados

# importar a base de dados
dataframe = pd.read_excel('Produtos.xlsx')

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [63]:
# passo 5: recalcular os precos

# atualizar as cotações 
dataframe.loc[dataframe['Moeda'] == 'Dólar', 'Cotação'] = float(dolar_value)
dataframe.loc[dataframe['Moeda'] == 'Euro', 'Cotação'] = float(euro_value)
dataframe.loc[dataframe['Moeda'] == 'Ouro', 'Cotação'] = float(gold_value)

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.160300,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.257133,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.160300,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.160300,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.257133,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.160300,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,290.900000,7000.00,1.15,8050.000


In [64]:
# atulizar o Preço de Compra = Preço Original * Cotação
dataframe['Preço de Compra'] = dataframe['Preço Original'] * dataframe['Cotação']

# atulizar o Preço de Venda = Preço de Compra * Margem
dataframe['Preço de Venda'] = dataframe['Preço de Compra'] * dataframe['Margem']

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.160300,5160.248397,1.40,7224.347756
1,Carro Renault,4500.00,Euro,5.257133,23657.098631,2.00,47314.197261
2,Notebook Dell,899.99,Dólar,5.160300,4644.218397,1.70,7895.171275
3,IPhone,799.00,Dólar,5.160300,4123.079700,1.70,7009.235490
4,Carro Fiat,3000.00,Euro,5.257133,15771.399087,1.90,29965.658265
5,Celular Xiaomi,480.48,Dólar,5.160300,2479.420944,2.00,4958.841888
6,Joia 20g,20.00,Ouro,290.900000,5818.000000,1.15,6690.700000


In [65]:
# passo 6: exportar a base de dados

# index=False remove a coluna com o índice de linha
dataframe.to_excel('Produtos-novo.xlsx', index=False)